# Recommendation based on top 10 sales items (whole)

In [2]:
import pandas as pd
df = pd.read_csv('D:/data_encoding_sel.csv')

In [4]:
def Rec_10(train,test):
    # Calculate best 10 sell in training
    D = {}
    for i in train['mod_itemfullname']:
        D[i]= len(train[train['mod_itemfullname']==i]['mod_itemfullname'])

    # Sort dictionary in descending order    
    import operator
    sorted_d = dict( sorted(D.items(), key=operator.itemgetter(1),reverse=True))

    # Get first 10 hightest brought items
    from itertools import islice
    H = list(islice(sorted_d.items(), 10))
    print('Top 10 highest brought (item,counts):\n',H)

    # Recommend list
    Rec_list = [i[0] for i in H]
    print('\nRecommend list:',Rec_list)

    # Recall@10
    rec10 = sum([1 for i in list(test['mod_itemfullname']) if i in Rec_list ])/len(list(test['mod_itemfullname']))
    print('Recall@10:',rec10)

# 80 train, 20 test
train = df[0:4836]
test = df[4836:]    
Rec_10(train,test)

Top 10 highest brought (item,counts):
 [(537, 103), (465, 94), (659, 82), (330, 76), (597, 76), (413, 73), (407, 71), (531, 67), (306, 66), (309, 57)]

Recommend list: [537, 465, 659, 330, 597, 413, 407, 531, 306, 309]
Recall@10: 0.10587262200165426


# Recommendation based on top 10 sales items (week)

In [5]:
from itertools import islice
import operator
df['user_id'] = pd.to_datetime(df['transaction_date']).apply(lambda time: time.dayofweek)

In [6]:
def Rec_week(train,test):    
    test_actual = list(test['mod_itemfullname'])
    
    def Rec_list(idx,Day):
        M = train[train['user_id']==idx]
        D = {}
        for i in M['mod_itemfullname']:
            D[i]= len(M[M['mod_itemfullname']==i]['mod_itemfullname'])

        # Sort dictionary in descending order    
        sorted_d = dict( sorted(D.items(), key=operator.itemgetter(1),reverse=True))

        # Get first 10 hightest brought items
        H = list(islice(sorted_d.items(), 10))
        #print('Top 10 highest brought (item,counts):\n',H)

        # Recommend list
        Rec_list = [i[0] for i in H]
        print('{} recommended list:'.format(Day),Rec_list)
        return Rec_list

    # Best 10 sales items for everday
    L = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    List_saved = {}
    for idx,Day in enumerate(L):
        List_saved[idx] = Rec_list(idx,Day)

    s = 0
    for i in range(len(test)):
        if test_actual[i] in List_saved[list(test['user_id'])[i]]:
            s+=1
    print('\nRecall@10: ',s/len(test_actual))

# 80 train, 20 test
train = df[0:4836]
test = df[4836:]
Rec_week(train,test)

Monday recommended list: [537, 330, 464, 306, 304, 413, 465, 407, 531, 309]
Tuesday recommended list: [537, 531, 659, 413, 330, 465, 407, 306, 466, 309]
Wednesday recommended list: [465, 330, 407, 145, 118, 306, 531, 413, 199, 309]
Thursday recommended list: [413, 407, 659, 199, 531, 306, 420, 465, 519, 146]
Friday recommended list: [597, 354, 465, 464, 717, 118, 309, 330, 642, 413]
Saturday recommended list: [597, 659, 465, 200, 330, 146, 717, 302, 199, 512]
Sunday recommended list: [659, 440, 120, 557, 125, 231, 134, 109, 95, 330]

Recall@10:  0.1141439205955335


# Recommendation based on top 10 sales items (time)

In [9]:
from datetime import datetime
df['user_id']= [0 for i in range(len(df))]

for i in range(len(df)):
    d_target = datetime.strptime(df['transaction_date'][i][11:], "%H:%M:%S")

    if datetime.strptime('08:00:00', "%H:%M:%S") <= d_target < datetime.strptime('11:00:00', "%H:%M:%S"):
        df['user_id'][i] = 0
    elif datetime.strptime('11:00:00', "%H:%M:%S") <= d_target < datetime.strptime('14:00:00', "%H:%M:%S"):
        df['user_id'][i] = 1 
    elif datetime.strptime('14:00:00', "%H:%M:%S") <= d_target < datetime.strptime('17:00:00', "%H:%M:%S"):
        df['user_id'][i] = 2 
    elif datetime.strptime('17:00:00', "%H:%M:%S") <= d_target < datetime.strptime('20:00:00', "%H:%M:%S"):
        df['user_id'][i] = 3 
    elif datetime.strptime('20:00:00', "%H:%M:%S") <= d_target < datetime.strptime('23:59:59', "%H:%M:%S"):
        df['user_id'][i] = 4
    elif datetime.strptime('00:00:00', "%H:%M:%S") <= d_target < datetime.strptime('08:00:00', "%H:%M:%S"):
        df['user_id'][i] = 5

C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\USER\anaconda3\lib\site-packages\ipyke

In [10]:
def Rec_time(train,test):    
    test_actual = list(test['mod_itemfullname'])
    
    def Rec_list(idx,Day):
        M = train[train['user_id']==idx]
        D = {}
        for i in M['mod_itemfullname']:
            D[i]= len(M[M['mod_itemfullname']==i]['mod_itemfullname'])

        # Sort dictionary in descending order    
        sorted_d = dict( sorted(D.items(), key=operator.itemgetter(1),reverse=True))

        # Get first 10 hightest brought items
        H = list(islice(sorted_d.items(), 10))
        #print('Top 10 highest brought (item,counts):\n',H)

        # Recommend list
        Rec_list = [i[0] for i in H]
        print('{} recommended list:'.format(Day),Rec_list)
        return Rec_list

    # Best 10 sales items for time
    L = ['08.00-11.00','11.00-14.00','14.00-17.00','17.00-20.00','20.00-00.00','00.00-08.00']
    List_saved = {}
    for idx,Day in enumerate(L):
        List_saved[idx] = Rec_list(idx,Day)

    s = 0
    for i in range(len(test)):
        if test_actual[i] in List_saved[list(test['user_id'])[i]]:
            s+=1
    print('\nRecall@10: ',s/len(test_actual))

# 80 train, 20 test
train = df[0:4836]
test = df[4836:]
Rec_time(train,test)

08.00-11.00 recommended list: [531, 118, 306, 200, 530, 304, 642, 9, 433, 520]
11.00-14.00 recommended list: [465, 464, 659, 309, 466, 537, 597, 413, 306, 354]
14.00-17.00 recommended list: [330, 407, 597, 145, 557, 200, 304, 353, 134, 413]
17.00-20.00 recommended list: [537, 659, 407, 330, 597, 38, 240, 199, 198, 121]
20.00-00.00 recommended list: [199, 330, 169, 174, 16, 717, 557, 21, 457, 456]
00.00-08.00 recommended list: [231, 413, 514, 306, 693, 531, 717, 304, 120, 695]

Recall@10:  0.14309346567411083


# Recommendation based on top 10 sales items (week and time)

In [11]:
df['user_id1'] = pd.to_datetime(df['transaction_date']).apply(lambda time: time.dayofweek)
df['user_id'] = [0 for i in range(len(df))]

In [13]:
for i in range(len(df['transaction_date'])):
    d_target = datetime.strptime(df['transaction_date'][i][11:], "%H:%M:%S")
    
    for j in range(7):
        if df['user_id1'][i]==j:  # Monday
            if datetime.strptime('08:00:00', "%H:%M:%S") <= d_target < datetime.strptime('11:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6
            elif datetime.strptime('11:00:00', "%H:%M:%S") <= d_target < datetime.strptime('14:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6+1 
            elif datetime.strptime('14:00:00', "%H:%M:%S") <= d_target < datetime.strptime('17:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6+2  
            elif datetime.strptime('17:00:00', "%H:%M:%S") <= d_target < datetime.strptime('20:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6+3  
            elif datetime.strptime('20:00:00', "%H:%M:%S") <= d_target < datetime.strptime('23:59:59', "%H:%M:%S"):
                df['user_id'][i] = j*6+4 
            elif datetime.strptime('00:00:00', "%H:%M:%S") <= d_target < datetime.strptime('08:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6+5       

C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py

In [14]:
def Rec_weektime(train,test):    
    test_actual = list(test['mod_itemfullname'])
    
    def Rec_list(idx,time):
        
        M = train[train['user_id']==idx]
        D = {}
        for i in M['mod_itemfullname']:
            D[i]= len(M[M['mod_itemfullname']==i]['mod_itemfullname'])

        # Sort dictionary in descending order    
        sorted_d = dict( sorted(D.items(), key=operator.itemgetter(1),reverse=True))

        # Get first 10 hightest brought items
        H = list(islice(sorted_d.items(), 10))
        #print('Top 10 highest brought (item,counts):\n',H)

        # Recommend list
        Rec_list = [i[0] for i in H]
        print('{} recommended list:'.format(time),Rec_list)
        return Rec_list

    T = [q for q in range(42)] 
    
    List_saved = {}
    for idx,time in enumerate(T):
        List_saved[idx] = Rec_list(idx,time)

    s = 0
    for i in range(len(test)):
        if test_actual[i] in List_saved[list(test['user_id'])[i]]:
            s+=1
    print('\nRecall@10: ',s/len(test_actual))

# 80 train, 20 test
train = df[0:4836]
test = df[4836:]
Rec_weektime(train,test)

0 recommended list: [531, 306, 75, 530, 100, 481, 413, 330, 304, 76]
1 recommended list: [464, 465, 466, 537, 309, 413, 420, 344, 58, 330]
2 recommended list: [304, 330, 306, 335, 407, 147, 512, 412, 531, 607]
3 recommended list: [537, 407, 145, 330, 659, 240, 519, 353, 199, 698]
4 recommended list: [174, 456, 38, 199, 594, 343, 407, 128, 309, 425]
5 recommended list: [231, 693, 413, 74, 530, 514, 575, 199, 567, 453]
6 recommended list: [531, 306, 118, 644, 694, 407, 75, 212, 530, 9]
7 recommended list: [537, 465, 659, 466, 464, 575, 694, 642, 412, 119]
8 recommended list: [537, 145, 134, 557, 353, 309, 330, 717, 51, 659]
9 recommended list: [537, 330, 531, 607, 407, 413, 198, 123, 38, 51]
10 recommended list: [513, 666, 200, 199, 457, 159, 585, 582, 407, 693]
11 recommended list: [413, 231, 304, 514, 531, 507, 145, 695, 693, 212]
12 recommended list: [306, 531, 118, 530, 105, 520, 200, 642, 598, 644]
13 recommended list: [465, 466, 309, 290, 146, 413, 185, 199, 407, 145]
14 recommende